In [1]:
import json
import os
from typing import List, Dict, Tuple
import numpy as np
from rank_bm25 import BM25Okapi
from datetime import datetime
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import jieba
import faiss
from pathlib import Path

/root/miniconda3/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-07-19 21:04:46.958120: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-19 21:04:47.015761: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-19 21:04:47.912990: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38

### 优化/修改 rag_v0 中的中文检索器

#### 修改内容

相较于 rag_v0 修改了如下方面：
1. 字符级分词 ---> 词语级分词
2. 稠密检索 (v0中需使用内存存储，重启后需重建索引，不适合大规模数据)
   1. 增加 FAISS 向量数据库方案
   2. 增加 Milvus 向量数据库方案
3. 独立BM25、FAISS、Bert检索类

#### 检索器流程

一般的 Retriever 流程如下：

1. 文本预处理: 分词(英文-空格, 中文-单个字符), 清洗(去噪-HTML标签/特殊字符)
2. 文本嵌入: 将文本转换为固定维度的向量
3. 检索算法:
   - 倒排索引(如 BM25)
   - 向量检索(如 向量数据库FAISS/Milvus, BERT)
   - 混合检索(结合向量检索和倒排索引, 初筛+精筛/综合得分)
4. 结果重排序
   - **相似度**：向量相似度
   - **模型**：排序模型（如 BERT-based ranking model）对候选文档进行排序
   - **混合排序**：多种排序方法 (如先按向量相似度排序, 再按关键词匹配度排序)

#### FAISS

##### 检索方法

在本实验中，采用 IndexFlatIP 作为检索方法：
1. 该方法是计算向量的 **内积** 作为度量方法
2. 内积结果受两个因素影响:
   1. 向量方向 (真正反映语义相似度)
   2. 向量长度 (与语义无关的干扰因素)
3. 实际处理中需进行归一化 --- 避免 **向量长度偏差** 问题，具体问题见下列代码块 **向量长度问题**
4. 在 FAISS 中可直接使用 `faiss.normalize_L2(embeddings)` 进行归一化
5. 注意存入参考文本时和查询时都须进行归一化

##### 向量长度问题

未归一化

In [4]:
vec_short = np.array([1, 1])  # 短文本嵌入
vec_long = np.array([2, 2])  # 长文本嵌入
query = np.array([1, 0])  # 参考文本嵌入

原始内积

In [5]:
print(f'短文本得分: {np.dot(query, vec_short)}')
print(f'长文本得分: {np.dot(query, vec_long)}')

短文本得分: 1
长文本得分: 2


- 结果：
  虽然长文本、短文本的夹角都和参考文本相同 (也就是余弦相似度应该相等)，但得到两个内积是不一样的，所以需要进行归一化

- 影响：
  1. 文本长度：长文本通常会产生更大的嵌入向量 (更多非0值)
  2. 系统会更倾向于返回更长的文档，而不是更相关的文档 (那么在 QA 中可能导致返回包含冗余信息的段落)

归一化

In [6]:
def normalize(v):
    return v / np.linalg.norm(v)


norm_short = normalize(vec_short)
norm_long = normalize(vec_long)
norm_query = normalize(query)

归一化后内积

In [7]:
print("短文本得分:", np.dot(norm_query, norm_short))
print("长文本得分:", np.dot(norm_query, norm_long))

短文本得分: 0.7071067811865475
长文本得分: 0.7071067811865475


归一化后的结果就相等了，文本的相似度不会受到向量长度的影响

主流框架的处理方式
1. FAISS: `faiss.normalize_L2(embeddings)`
2. Milvus: 内置 `metric_type="IP"` 会自动归一化
3. Elasticsearch: `cosineSimilarity` 函数自动处理归一化
4. SentenceTransformers: 默认返回已归一化的嵌入

#### 中文检索器类

##### 基础类

In [2]:
class ChineseRetriever:
    """
    基础类: 提供文档加载和结果返回的函数，包含以下函数
    _load_json_docs: 加载并预处理JSON文档
    get_document: 根据索引获取完整文档
    print_results: 格式化打印检索结果
    """

    def __init__(self, json_path: str):
        """
        初始化中文检索器
        :param json_path: JSON文件路径, 格式为[{'question':'', 'answer':''}, ...]
        """
        self.docs = self._load_json_docs(json_path)
        self.questions = [doc['question'] for doc in self.docs]
        self.answers = [doc['answer'] for doc in self.docs]

    def _load_json_docs(self, path: str) -> List[Dict]:
        """加载并预处理JSON文档"""
        with open(path, 'r', encoding='utf-8') as f:
            docs = json.load(f)

        # 中文分词处理(按词语级分割)
        for doc in docs:
            doc['tokenized'] = list(jieba.cut(doc['question']))  # 按jieba的默认词典进行分词
        return docs

    def get_document(self, index: int) -> Dict:
        """根据索引获取完整文档"""
        return self.docs[index]

    def print_results(self, results: List[Tuple[int, float]]):
        """格式化打印检索结果"""
        for rank, (idx, score) in enumerate(results, 1):
            doc = self.get_document(idx)
            print(f"Rank {rank} (Score: {score:.4f}):")
            print(f"Q: {doc['question']}")
            print(f"A: {doc['answer']}\n")


##### BM25

In [3]:
class BM25Retriever(ChineseRetriever):
    def __init__(self, json_path: str):
        """
        初始化BM25检索器
        :param json_path: JSON文件路径, 格式为[{'question':'', 'answer':''}, ...]
        """
        super().__init__(json_path)

        # 初始化稀疏检索(BM25)
        self.bm25 = self._init_sparse_retriever()

    def _init_sparse_retriever(self):
        """初始化BM25稀疏检索器"""
        tokenized_corpus = [doc['tokenized'] for doc in self.docs]
        return BM25Okapi(tokenized_corpus)

    def sparse_retrieve(self, query: str, top_k: int = 5) -> List[Tuple[int, float]]:
        """
        BM25稀疏检索
        :return: 返回(top_k个(文档索引, 得分))列表, 如[(3, 8.21), (1, 6.54)...], 按得分降序排序
        :param query: 原始查询字符串
        :process: 
            1. 先将中文字符串转换为字符列表 (即进行文本分割), BM25基于词频统计, 中文无空格因此需主动分词, 目前仅使用字符级分词, 后续可尝试分词工具, 如 jieba
            2. 再计算BM25得分
            3. 最后返回前k个文档
        """
        # 同样修改成使用jieba默认词典分词
        tokenized_query = list(jieba.cut(query))
        # BM25计算公式
        scores = self.bm25.get_scores(tokenized_query)
        # 前k个文档的索引
        top_indices = np.argsort(scores)[-top_k:][::-1]
        '''
        return [(i, scores[i]) for i in top_indices]
        上述代码在当前环境 (当前环境见 requirements.txt) 中会报错如下：
        Expression of type "list[tuple[NDArray[intp], Any]]" cannot be assigned to return type "List[Tuple[int, float]]"
        NDArray[intp] 说明：
            1. NDArray: NumPy数组, 表示一个多维数组, 是 numpy.ndarray 的别名
            2. intp: 32位系统 --- 32位整数; 64位系统 --- 64位整数
        '''
        return [(int(i), scores[int(i)]) for i in top_indices]


##### Bert

In [4]:
class BertRetriever(ChineseRetriever):
    def __init__(self,
                 json_path: str,
                 model_path_or_name: str = 'bert-base-chinese'
                 ):
        """
        初始化中文检索器
        :param json_path: JSON文件路径, 格式为[{'question':'', 'answer':''}, ...]
        :param model_path_or_name: 稠密检索模型名称 or 路径(默认bert-base-chinese)
        """
        super().__init__(json_path)

        # 初始化稠密检索(BERT)
        self.encoder = SentenceTransformer(model_path_or_name)
        self.doc_embeddings = self._init_dense_retriever()

    def _init_dense_retriever(self) -> np.ndarray:
        """预计算所有文档的BERT嵌入"""
        return self.encoder.encode(self.questions)

    def dense_retrieve(self, query: str, top_k: int = 5) -> List[Tuple[int, float]]:
        """
        BERT稠密检索
        :return: 返回(top_k个(文档索引, 余弦相似度))列表
        :param query: 原始查询字符串
        :process: 
            1. 分词器将文本转换为子词单元
            2. 添加[CLS]/[SEP]等特殊标记
            3. 通过12层Transformer获取[CLS]位置的768维向量
        """
        # 编码
        query_embedding = self.encoder.encode(query)
        # 余弦相似度
        similarities = cosine_similarity(
            [query_embedding],
            self.doc_embeddings
        )[0]
        # .argsort 顺序
        top_indices = np.argsort(similarities)[-top_k:][::-1]

        '''
        return [i, similarities[i]) for i in top_indices]
        上述代码在当前环境 (当前环境见 requirements.txt) 中会报错如下：
        Expression of type "list[tuple[NDArray[intp], Any]]" cannot be assigned to return type "List[Tuple[int, float]]"
        NDArray[intp] 说明：
            1. NDArray: NumPy数组, 表示一个多维数组, 是 numpy.ndarray 的别名
            2. intp: 32位系统 --- 32位整数; 64位系统 --- 64位整数
        '''
        return [(int(i), similarities[int(i)]) for i in top_indices]


##### FAISS

In [20]:
class FaissRetriever(ChineseRetriever):
    def __init__(self,
                 json_path: str,
                 db_dir: str = 'vector_db',
                 model_path_or_name: str = 'bert-base-chinese'
                 ):
        """
        初始化中文检索器
        :param json_path: JSON文件路径, 格式为[{'question':'', 'answer':''}, ...]
        :param model_path_or_name: 稠密检索模型名称 or 路径(默认bert-base-chinese)
        """
        super().__init__(json_path=json_path)

        # 初始化稠密检索(BERT)
        self.encoder = SentenceTransformer(model_path_or_name)
        self.doc_embeddings = self._init_dense_retriever()

        # 初始化稠密检索(FAISS)
        self.db_dir = Path(db_dir)
        self.db_dir.mkdir(exist_ok=True)

        # 初始化检索系统
        self._init_faiss()

    def _init_dense_retriever(self) -> np.ndarray:
        """预计算所有文档的BERT嵌入"""
        return self.encoder.encode(self.questions)

    def _init_faiss(self):
        """初始化FAISS向量数据库"""
        db_path = self.db_dir / 'faiss.index'
        if db_path.exists():
            self.faiss_index = faiss.read_index(str(db_path))
        else:
            # 将numpy数组转换为FAISS需要的格式
            vec = self.doc_embeddings.astype('float32')
            '''
            构建索引:
            1. 由于文本相似度的衡量一般使用余弦相似度，那么可以通过 单位向量内积=余弦相似度
            2. 单位向量，即L2范数=1的向量
            3. 选用IndexFlatIP: IP是指内积（Inner Product）, 对向量进行L2正则化之后再内积就相当于计算余弦相似度
            '''
            # 这里需传入一个向量的维度，从而创建一个空的索引
            self.faiss_index = faiss.IndexFlatIP(vec.shape[1])

            # 归一化
            faiss.normalize_L2(vec)

            '''
            如果想自定义 id，需要注意: IndexFlatIP 创建映射时不支持 add_with_ids
            因此在创建 Index 时需要传入一个 IndexIDMap
            然后将 向量嵌入vec 和 xids 一起加入到 FAISS 中
            '''
            # xids = np.array([i for i in range(1, vec.shape[0] + 1)])
            # IDMap_index = faiss.IndexIDMap(self.faiss_index)
            # IDMap_index.add_with_ids(vec, xids)

            # 将向量数据加入索引
            # 当前版本需要传入 vec.shape[0] 获取向量数量
            # 否则会报错 Argument missing for parameter "x"
            # e.g. self.faiss_index.add(vec.shape[0], vec)
            ##############################################
            # faiss-cpu 1.10.0 版本无需添加 vec.shape[0]
            self.faiss_index.add(vec)

            # 最后保存索引
            faiss.write_index(self.faiss_index, str(db_path))

    def faiss_retrieve(self,
                       query: str,
                       top_k: int = 5,
                       ) -> List[Tuple[int, float]]:
        """
        FAISS 向量数据库检索

        Args:
            query (str): 原始查询字符串
            top_k (int, optional): 选取前 K 个文档. Defaults to 5.

        Returns:
            List[Tuple[int, float]]: 包含(top_k个(文档索引, 相似度得分))的列表，
                                    得分范围[-1,1]，1表示完全相似
                                    示例: [(0, 0.92), (2, 0.85), ...]
        """
        # 编码
        query_embedding = self.encoder.encode(query).astype('float32')

        # 确保向量是2D数组 (1, embedding_dim)
        if len(query_embedding.shape) == 1:
            query_embedding = query_embedding.reshape(1, -1)

        # 归一化 (使得内积等价于余弦相似度)
        faiss.normalize_L2(query_embedding)

        # FAISS 检索 (返回形状 (1, top_k))
        scores, indices = self.faiss_index.search(
            query_embedding,
            top_k
        )

        # (编号, 得分)
        # if idx != -1: 过滤无效匹配项
        results = [(int(idx), float(score)) for idx, score in zip(indices[0], scores[0]) if idx != -1]
        return results


##### Hybrid

In [24]:
class HybridRetriever(ChineseRetriever):
    """混合检索器类，组合稀疏检索和稠密检索"""

    def __init__(self,
                 json_path: str,
                 weight: float = 0.6,
                 model_path_or_name: str = 'bert-base-chinese'
                 ):
        """
        初始化混合检索器
        :param json_path: JSON文件路径, 格式为[{'question':'', 'answer':''}, ...]
        :param weight: 稠密检索的权重（0-1之间）
        #
        :param bert_retriever: 稠密检索器 BertRetriever 实例
        :param faiss_retriever: 稠密检索器 FaissRetriever 实例
        :param bm25_retriever: 稀疏检索器 BM25Retriever 实例
        """
        super().__init__(json_path)
        self.weight = weight
        self.model_path_or_name = model_path_or_name

        self.bert_retriever = BertRetriever(json_path=json_path, model_path_or_name=self.model_path_or_name)
        self.faiss_retriever = FaissRetriever(json_path=json_path, model_path_or_name=self.model_path_or_name)
        self.bm25_retriever = BM25Retriever(json_path=json_path)

    def bm25_bert_retrieve(self, query: str, top_k: int = 5, dense_weight: float = None) -> List[Tuple[int, float]]:
        """
        混合检索(默认稠密权重0.6, 稀疏权重0.4)
        :param dense_weight: 稠密检索得分权重
        :return: 返回(top_k个(文档索引, 综合得分))列表
        """
        # 在未传值时使用self.weight
        if dense_weight is None:
            dense_weight = self.weight

        # 各自检索2倍于最终需要的文档量, 扩大候选池
        bm25_results = dict(self.bm25_retriever.sparse_retrieve(query, top_k * 2))
        bert_results = dict(self.bert_retriever.dense_retrieve(query, top_k * 2))

        # 归一化得分
        # BM25得分范围[0, +∞]
        max_sparse = max(bm25_results.values()) if bm25_results else 1
        # bert 余弦相似度[-1, 1]
        max_dense = max(bert_results.values()) if bert_results else 1

        # 融合得分
        fused_scores = {}
        all_indices = set(bm25_results.keys()) | set(bert_results.keys())
        for idx in all_indices:
            norm_sparse = bm25_results.get(idx, 0) / max_sparse
            norm_dense = bert_results.get(idx, 0) / max_dense
            fused_scores[idx] = dense_weight * norm_dense + (1 - dense_weight) * norm_sparse

        # 返回top_k结果
        top_indices = sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
        return top_indices

    def bm25_faiss_retrieve(self, query: str, top_k: int = 5, dense_weight: float = None) -> List[Tuple[int, float]]:
        """
        混合检索(默认稠密权重0.6, 稀疏权重0.4)
        :param dense_weight: 稠密检索得分权重
        :return: 返回(top_k个(文档索引, 综合得分))列表
        """
        # 在未传值时使用self.weight
        if dense_weight is None:
            dense_weight = self.weight

        # 各自检索2倍于最终需要的文档量, 扩大候选池
        bm25_results = dict(self.bm25_retriever.sparse_retrieve(query, top_k * 2))
        faiss_results = dict(self.faiss_retriever.faiss_retrieve(query, top_k * 2))

        # 归一化得分
        # BM25得分范围[0, +∞]
        max_sparse = max(bm25_results.values()) if bm25_results else 1
        # faiss 内积[-1, 1]
        max_dense = max(faiss_results.values()) if faiss_retriever else 1

        # 融合得分
        fused_scores = {}
        all_indices = set(bm25_results.keys()) | set(faiss_results.keys())
        for idx in all_indices:
            norm_sparse = bm25_results.get(idx, 0) / max_sparse
            norm_dense = faiss_results.get(idx, 0) / max_dense
            fused_scores[idx] = dense_weight * norm_dense + (1 - dense_weight) * norm_sparse

        # 返回top_k结果
        top_indices = sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
        return top_indices


##### python 路径操作

传统路径操作如下

In [ ]:
path = os.path.join('data', 'index.txt')

现代Path方式
1. 与传统方式等效
2. Path('data') 创建Path对象
3. / 运算符重载为路径拼接
4. 自动处理不同OS的路径分隔符 (Linux:/ Windows:\\)

In [ ]:
# path = Path('data') / 'index.txt'

#### 测试

In [7]:
# 测试查询
query = "注册公司有哪些注意事项"

##### BM25

In [8]:
# 初始化检索器
bm25Retriever = BM25Retriever(json_path="QA_公司法.json")

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.588 seconds.
Prefix dict has been built successfully.


In [9]:
print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("=" * 40 + "\n稀疏检索结果(BM25):")
bm25_results = bm25Retriever.sparse_retrieve(query=query, top_k=3)
bm25Retriever.print_results(bm25_results)

2025-07-19 21:05:28
稀疏检索结果(BM25):
Rank 1 (Score: 22.4545):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 3、费用

Rank 2 (Score: 22.4545):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 企业按组建形式可以分为有限公司、个人独资企业、合伙企业。目前，90%以上的企业类型为有限公司(以注册资本承担对外赔偿限额)，而个人独资企业或合伙企业因投资者承担无限责任而选择这2种企业类型的较少。

Rank 3 (Score: 22.4545):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 首先，办理公司注册登记，需特别注意的是在公司经营范围中需加上“从事货物及技术的进出口业务”这一条。有了这条经营范围就才能申请进出口备案。从事进出口业务的也写清楚具体的业务范围。其次，在公司注册完毕及银行开户之后，申请进出口备案。进出口备案包括海关、电子口岸、外汇、检验检疫等备案手续。


##### 稠密检索(BERT)

In [10]:
# 初始化检索器
bertRetriever = BertRetriever(json_path="QA_公司法.json", model_path_or_name='/root/autodl-tmp/data/models/bert-base-chinese')

No sentence-transformers model found with name /root/autodl-tmp/data/models/bert-base-chinese. Creating a new one with mean pooling.


In [11]:
print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("=" * 40 + "\n稠密检索结果(BERT):")
bert_results = bertRetriever.dense_retrieve(query=query, top_k=3)
bertRetriever.print_results(bert_results)

2025-07-19 21:05:58
稠密检索结果(BERT):
Rank 1 (Score: 0.9319):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 5、进出口权

Rank 2 (Score: 0.9319):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 按照《公司法》的规定，有限公司最低注册资本为3万元人民币，其中，一人有限公司最低注册资本为10万元人民币。注册资本可以分期出资，首批不低于20%，其余注册资本可在2年内到位。但是，不同对于最低注册资本的要求是不一样的。例如，国际货运代理公司要求最低注册资本为500元人民币。需注意的是，任何公司在设立登记时，除了要符合公司法对注册资本的要求，也要符合行业法规对最低注册资本的规定。在还需符合外资企业法律法规对于注册资本的要求

Rank 3 (Score: 0.9319):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 首先，办理公司注册登记，需特别注意的是在公司经营范围中需加上“从事货物及技术的进出口业务”这一条。有了这条经营范围就才能申请进出口备案。从事进出口业务的也写清楚具体的业务范围。其次，在公司注册完毕及银行开户之后，申请进出口备案。进出口备案包括海关、电子口岸、外汇、检验检疫等备案手续。


##### 稠密检索(FAISS)

In [21]:
faiss_retriever = FaissRetriever(json_path='QA_公司法.json', model_path_or_name='/root/autodl-tmp/data/models/bert-base-chinese')

No sentence-transformers model found with name /root/autodl-tmp/data/models/bert-base-chinese. Creating a new one with mean pooling.


In [22]:
print("=" * 40 + "\n稠密检索结果(FAISS):")
faiss_results = faiss_retriever.faiss_retrieve(query=query, top_k=3)
faiss_retriever.print_results(faiss_results)

稠密检索结果(FAISS):
Rank 1 (Score: 0.9319):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 首先，办理公司注册登记，需特别注意的是在公司经营范围中需加上“从事货物及技术的进出口业务”这一条。有了这条经营范围就才能申请进出口备案。从事进出口业务的也写清楚具体的业务范围。其次，在公司注册完毕及银行开户之后，申请进出口备案。进出口备案包括海关、电子口岸、外汇、检验检疫等备案手续。

Rank 2 (Score: 0.9319):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 企业按组建形式可以分为有限公司、个人独资企业、合伙企业。目前，90%以上的企业类型为有限公司(以注册资本承担对外赔偿限额)，而个人独资企业或合伙企业因投资者承担无限责任而选择这2种企业类型的较少。

Rank 3 (Score: 0.9319):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 5、进出口权


##### 混合检索

In [25]:
hybrid_retriever = HybridRetriever(json_path='QA_公司法.json', model_path_or_name='/root/autodl-tmp/data/models/bert-base-chinese')

No sentence-transformers model found with name /root/autodl-tmp/data/models/bert-base-chinese. Creating a new one with mean pooling.
No sentence-transformers model found with name /root/autodl-tmp/data/models/bert-base-chinese. Creating a new one with mean pooling.


In [26]:
print("=" * 40 + "\n混合检索结果(bm25 + bert):")
bm25_bert_results = hybrid_retriever.bm25_bert_retrieve(query=query, top_k=3)
hybrid_retriever.print_results(bm25_bert_results)

混合检索结果(bm25 + bert):
Rank 1 (Score: 1.0000):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 对于贸易公司或进出口公司来说，基本上都要申请一般纳税人资格(开具增值税专用发票)。各个区或同一个区的不同税务所对于企业申请一般纳税人资格的要求或规定是有些差异的

Rank 2 (Score: 1.0000):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 首先，办理公司注册登记，需特别注意的是在公司经营范围中需加上“从事货物及技术的进出口业务”这一条。有了这条经营范围就才能申请进出口备案。从事进出口业务的也写清楚具体的业务范围。其次，在公司注册完毕及银行开户之后，申请进出口备案。进出口备案包括海关、电子口岸、外汇、检验检疫等备案手续。

Rank 3 (Score: 1.0000):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 2、要求


In [27]:
print("=" * 40 + "\n混合检索结果(bm25 + faiss):")
bm25_faiss_results = hybrid_retriever.bm25_faiss_retrieve(query=query, top_k=3)
hybrid_retriever.print_results(bm25_faiss_results)

混合检索结果(bm25 + faiss):
Rank 1 (Score: 1.0000):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 首先，办理公司注册登记，需特别注意的是在公司经营范围中需加上“从事货物及技术的进出口业务”这一条。有了这条经营范围就才能申请进出口备案。从事进出口业务的也写清楚具体的业务范围。其次，在公司注册完毕及银行开户之后，申请进出口备案。进出口备案包括海关、电子口岸、外汇、检验检疫等备案手续。

Rank 2 (Score: 1.0000):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 企业按组建形式可以分为有限公司、个人独资企业、合伙企业。目前，90%以上的企业类型为有限公司(以注册资本承担对外赔偿限额)，而个人独资企业或合伙企业因投资者承担无限责任而选择这2种企业类型的较少。

Rank 3 (Score: 1.0000):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 3、费用
